# Sammensetning av alt

#### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plts
from sklearn.metrics import accuracy_score, precision_score, recall_score
from LSTM import long_short_term_memory as LSTM
from rf import rf

#### Colors

In [2]:
rsrp_colors = ['#384959', '#6A89A7', '#88BDF2', '#BDDDFC']
rsrq_colors = ['#614419', '#B37E2E', '#DB9A39', '#FFB343']
sinr_colors = ['#135E4B', '#4CB572', '#A1D8B5', '#CCDCDB']

In [3]:
'''
    Reading the dataset and returning two pandas DataFrames:
        The first DataFrame containing every datapoint belonging to the given campaign (if any)
        The second DataFrame containing only one column - the mode (4G or 5G)
'''
def read_dataset(filename, campaign=None, features=None):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.loc[:, ~df.columns.str.match('Timestamp')]
    df = df.replace('DC', 1)
    df = df.replace('LTE', 0)
    df = df[df['campaign'].str.contains('Driving')]

    if campaign != None:
        df = df[df['campaign'] == campaign]
    
    df, mode = df[features], df['Mode']
    return df, mode

## LSTM-prediction of signal strength indicators

In [4]:
campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

rsrp_results = {}
feature_selection = [['RSRP'], 
                     ['RSRP', 'RSRQ'], ['RSRP', 'SINR'], ['RSRP', 'SINR', 'RSRQ'],
                     ['RSRP', 'SSS_RSRP'], 
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    features.append('campaign')
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    rsrp_results[used_features] = LSTM(X=X, target_feature='RSRP', features=feats, n_steps_in=5, n_steps_out=1)

rsrq_results = {}
feature_selection = [['RSRQ'], 
                     ['RSRQ', 'SINR'], ['RSRQ', 'RSRP'], ['RSRQ', 'RSRP', 'SINR'],
                     ['RSRQ', 'SSS_RSRQ'], 
                     ['RSRQ', 'RSRP', 'SINR', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    features.append('campaign')
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    rsrq_results[used_features] = LSTM(X=X, target_feature='RSRQ', features=feats, n_steps_in=5, n_steps_out=1)

sinr_results = {}
feature_selection = [['SINR'], 
                     ['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['SINR', 'RSRP', 'RSRQ'],
                     ['SINR', 'SSS_SINR'], 
                     ['SINR', 'RSRP', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    features.append('campaign')
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    sinr_results[used_features] = LSTM(X=X, target_feature='SINR', features=feats, n_steps_in=5, n_steps_out=2)

In [5]:
df_rsrp_results = pd.DataFrame.from_dict(rsrp_results, orient='index')

df_rsrp_results.reset_index(inplace=True)
df_rsrp_results.rename(columns={'index': 'Combination'}, inplace=True)
df_rsrp_results = df_rsrp_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_rsrp_results

,Combination,loss_final,val_loss_final,mad
0,RSRP,0.00059,0.00057,6.5
1,"RSRP, RSRQ",0.00058,0.00054,6.4
2,"RSRP, SINR",0.00060,0.00058,6.4
3,"RSRP, SINR, RSRQ",0.00060,0.00066,6.2
4,"RSRP, SSS_RSRP",0.00058,0.00108,6.2
5,"RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ",0.00064,0.00093,6.2


In [6]:
df_rsrq_results = pd.DataFrame.from_dict(rsrq_results, orient='index')

df_rsrq_results.reset_index(inplace=True)
df_rsrq_results.rename(columns={'index': 'Combination'}, inplace=True)
df_rsrq_results = df_rsrq_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_rsrq_results

,Combination,loss_final,val_loss_final,mad
0,RSRQ,0.00157,0.00107,3.7
1,"RSRQ, SINR",0.00167,0.00098,3.7
2,"RSRQ, RSRP",0.00154,0.00098,3.8
3,"RSRQ, RSRP, SINR",0.00158,0.00138,3.8
4,"RSRQ, SSS_RSRQ",0.00151,0.00109,3.7
5,"RSRQ, RSRP, SINR, SSS_RSRP, SSS_SINR, SSS_RSRQ",0.00153,0.00115,3.7


In [7]:
df_sinr_results = pd.DataFrame.from_dict(sinr_results, orient='index')

df_sinr_results.reset_index(inplace=True)
df_sinr_results.rename(columns={'index': 'Combination'}, inplace=True)
df_sinr_results = df_sinr_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_sinr_results

,Combination,loss_final,val_loss_final,mad
0,SINR,0.00156,0.00119,6.0
1,"SINR, RSRQ",0.00156,0.00127,6.2
2,"SINR, RSRP",0.00155,0.00105,6.2
3,"SINR, RSRP, RSRQ",0.00154,0.00115,6.3
4,"SINR, SSS_SINR",0.00167,0.00149,6.0
5,"SINR, RSRP, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ",0.00154,0.00187,6.2


In [8]:
campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

results = {}
target_features = ['RSRP', 'SINR', 'RSRQ']
feature_selection = [['SINR'], ['RSRP'], ['RSRQ'], 
                     ['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['SINR', 'RSRP', 'RSRQ'],
                     ['SINR', 'SSS_SINR'], ['RSRP', 'SSS_RSRP'], ['RSRQ', 'SSS_RSRQ'], 
                     ['SINR', 'RSRP', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

for target in target_features:
    for features in feature_selection:
        if features.__contains__(target):
            used_features = ', '.join(features)
            used_features = used_features + '-' + target
            feats = features.copy()
            features.append('campaign')
            X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
            if feats.__contains__('campaign'):
                feats.remove('campaign')
            print(feats)
            results[used_features] = LSTM(X=X, target_feature=target, features=feats, n_steps_in=5, n_steps_out=1)

['RSRP']
['SINR', 'RSRP']
['RSRP', 'RSRQ']
['SINR', 'RSRP', 'RSRQ']
['RSRP', 'SSS_RSRP']
['SINR', 'RSRP', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']
['SINR']
['SINR', 'RSRQ']
['SINR', 'RSRP']


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_results = pd.DataFrame.from_dict(results, orient='index')

df_results.reset_index(inplace=True)
df_results.rename(columns={'index': 'Combination'}, inplace=True)
df_results = df_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_results

## Classification of mode using Random Forest

In [14]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ'],
                     ['RSRP', 'SSS_RSRP'], ['SINR', 'SSS_SINR'], ['RSRQ', 'SSS_RSRQ'],
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

rf_results = {}

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    rf_results[used_features] = rf(X, mode)

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df

,Combination,Accuracy,Precision,Recall
0,"SINR, RSRQ",0.928319,0.960465,0.767658
1,"SINR, RSRP",0.920747,0.942478,0.764811
2,"RSRP, RSRQ",0.901060,0.935780,0.708333
3,"RSRP, SINR, RSRQ",0.917214,0.945946,0.750000
4,"RSRP, SSS_RSRP",0.972236,0.954463,0.945848
5,"SINR, SSS_SINR",0.968198,0.955473,0.929603
6,"RSRQ, SSS_RSRQ",0.957092,0.907057,0.944444
7,"RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ",0.975265,0.931937,0.981618


In [ ]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ']]

rf_results = {}

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    metrics = rf(X, mode)
    # metrics['pred_y_pred'] = metrics['model'].predict()

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df